# Linear Regression Consulting Project

## Project objective:
Predict how many crew members a ship will need

## Project detail:

Hyundai Heavy Industries wants to build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

#### hint: use StringIndexer or OneHotEncoder

##### StringIndexer:
StringIndexer (equivalent to LabelEncoder in sklearn) assigns index [0-num_labels] to each string.
The problem here is, since there are different numbers in the same column, the model will misunderstand the data to be in some kind of order. But this is not the case at all. To overcome this problem, we use One Hot Encoder.

##### OneHotEncoder:
**Note:** pyspark OneHotEncoder is different from scikit-learn’s OneHotEncoder.

A one-hot encoder that maps a column of **category indices** to a column of binary vectors. So, we should encode categorical features to categorical indices using **StringIndexer** first then apply pyspark OneHotEncoder.

**Note:** The last category is not included by default (configurable via dropLast), because it makes the vector entries sum up to one, and hence linearly dependent. So an input value of 4.0 maps to [0.0, 0.0, 0.0, 0.0].


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MLlibLRProject').getOrCreate()


In [2]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('cruise_ship_info.csv',header=True, inferSchema=True)

In [6]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [7]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

In [20]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_lineIndex")
indexed = indexer.fit(data).transform(data)
indexed.head(2)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_lineIndex=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_lineIndex=16.0)]

In [21]:
onehotencoder = OneHotEncoder(inputCol="Cruise_lineIndex",
                        outputCol="Cruise_lineVec")
data_encoded = onehotencoder.fit(indexed).transform(indexed)
data_encoded.head(2)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_lineIndex=16.0, Cruise_lineVec=SparseVector(19, {16: 1.0})),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_lineIndex=16.0, Cruise_lineVec=SparseVector(19, {16: 1.0}))]

In [22]:
data_encoded.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_lineIndex: double (nullable = false)
 |-- Cruise_lineVec: vector (nullable = true)



In [26]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density',
                                       'Cruise_lineVec'],
                            outputCol = 'features')
final_data = assembler.transform(data_encoded)
final_data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_lineIndex: double (nullable = false)
 |-- Cruise_lineVec: vector (nullable = true)
 |-- features: vector (nullable = true)



In [27]:
#train test split
train_data , test_data = final_data.randomSplit([0.7,0.3])

In [40]:
# Linear regression model
lr = LinearRegression(featuresCol='features',labelCol='crew',predictionCol='prediction',regParam=0.0, elasticNetParam=0.0)
model = lr.fit(train_data)
test_results = model.evaluate(test_data)

In [41]:
test_results.rootMeanSquaredError


1.221860194595286

In [32]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -0.4475393721193264|
|  -1.374244357014537|
| -0.1632993384882635|
|  0.1212798742650758|
| 0.01745069143771527|
|  6.9873043559784485|
|  1.0081908975550498|
| -1.6859014051563053|
|  0.4565456708653848|
|-0.17532179889178678|
|     0.5768974665908|
| 0.43922773189591346|
|-0.24919394911010428|
|-0.16690026738364594|
| -0.4918077937428844|
| -0.0474860369023169|
|-0.05493153615804047|
|  0.2227554821639277|
| 0.13237259729023698|
|  0.4963341391585736|
+--------------------+
only showing top 20 rows

